In [1]:
%matplotlib inline
import editdistance
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
from glob import glob

In [3]:
landscapes=glob("data/TF_binding_landscapes/*")

In [12]:
def is_sequence_a_peak(model,sequence,alphabet="AGTC"):
    neighbor=[s for s in sequence]
    sequence_fitness=model.get_fitness(sequence)
    for position in range(len(sequence)):
        for aa in alphabet:
            if aa!=sequence[position]:
               neighbor[position]=aa
               neighbor_string="".join(neighbor)
               if sequence_fitness<model.get_fitness(neighbor_string):
                  return 0
               neighbor=[s for s in sequence]
    return 1

In [5]:
from models.Ground_truth_oracles.TF_binding_landscape_models import TF_binding_landscape

In [6]:
model=TF_binding_landscape()

In [20]:
model.construct(landscapes[10])

In [21]:
len(model.sequences)

65536

In [29]:
all_seqs={}
for landscape in landscapes:
    model=TF_binding_landscape()
    model.construct(landscape)
    max_peak=-1
    current_max="AAAAAAAA"
    for seq in model.sequences:
        if not seq in all_seqs:
           all_seqs[seq]=[0,[]]
        if is_sequence_a_peak(model,seq):
           all_seqs[seq][0]+=1
           if model.get_fitness(seq)>max_peak:
              max_peak = model.get_fitness(seq)
              current_max= seq
    all_seqs[current_max][1].append(landscape[landscape.index("data/TF_binding_landscapes/"):landscape.index(".txt")])
        

In [40]:
count=0

for k in sorted(all_seqs, key=all_seqs.get, reverse=False):
    count+=1
    print(k, all_seqs[k])
    if count>10:
        break

AAAAAAAT [0, []]
ATTTTTTT [0, []]
AAAAAACG [0, []]
CGTTTTTT [0, []]
AAAAAACT [0, []]
AGTTTTTT [0, []]
AAAAAATG [0, []]
CATTTTTT [0, []]
AAAAACCC [0, []]
GGGTTTTT [0, []]
AAAAACCG [0, []]


In [41]:
count=0

for k in sorted(all_seqs, key=all_seqs.get, reverse=True):
    count+=1
    print(k, all_seqs[k])
    if count>10:
        break

GCTCGAGC [106, []]
CACGCGTC [103, []]
GACGCGTG [103, []]
ATTATAAT [95, []]
CCCCGCCC [93, ['data/TF_binding_landscapes/EGR2_R409W_R1_8mers', 'data/TF_binding_landscapes/KLF1_REF_R1_8mers', 'data/TF_binding_landscapes/KLF1_H299Y_R1_8mers', 'data/TF_binding_landscapes/KLF1_REF_R2_8mers']]
GGGCGGGG [93, []]
TTAATTAA [91, ['data/TF_binding_landscapes/MSX2_R172H_R1_8mers', 'data/TF_binding_landscapes/ARX_R332H_R1_8mers', 'data/TF_binding_landscapes/HOXC4_REF_R2_8mers', 'data/TF_binding_landscapes/ISX_R83Q_R1_8mers', 'data/TF_binding_landscapes/POU3F4_V322M_R1_8mers', 'data/TF_binding_landscapes/HOXC4_N178S_R2_8mers', 'data/TF_binding_landscapes/HOXC4_R158L_R1_8mers', 'data/TF_binding_landscapes/PAX4_R183C_R1_8mers', 'data/TF_binding_landscapes/ARX_REF_R1_8mers', 'data/TF_binding_landscapes/POU3F4_A312V_R1_8mers', 'data/TF_binding_landscapes/VSX2_R200Q_R1_8mers', 'data/TF_binding_landscapes/MSX2_P148H_R2_8mers', 'data/TF_binding_landscapes/PITX2_R130W_R1_8mers', 'data/TF_binding_landscapes/AR

In [48]:
selected_sequences=[ 'TTAATTAA',   #  often global peak
                     'GCTCGAGC','GCGCGCGC','TGCGCGCC','ATATAGCC','GTTTGGTA',"ATTATGTT",  #  often local peaks
                    "CAGTTTTT","AAAAATTT", "AAAAACGC", "GTTGTTTT", "TGCTTTTT","AAAGATAG","CCTTCTTT","AAAGAGAG"] # never local peaks 

In [52]:
with open("TF_starting_id.csv","w") as outfile:
    count=0
    outfile.write("id,start,local_peak_in_num_landscapes,global_peak_in_landscapes\n")
    
    for seq in selected_sequences:
        if len(all_seqs[seq][1])>0:
            last_col=";".join([x.strip("data/TF_binding_landscapes/").strip("_8mers") for x in all_seqs[seq][1]])
        else:
            last_col=""
        outfile.write(f'TF{count},{seq},{all_seqs[seq][0]},{last_col}\n')
        count+=1